First let's generate a few random points on a 2D plane.

We have to import a few things, namely:
- numpy for generating random numbers
- matplotlib.pyplot for plotting
- cv2 for image processing
- io for reading and writing files into memory
- scipy.spatial for calculating the Voronoi diagram

In [ ]:
import cv2
import io
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d

In [ ]:
def generate_image(img: io.BytesIO):
    # Generate random points
    points = np.random.rand(10, 2)

    # Create the figure
    fig = plt.figure(figsize=(5, 5))

    # Turn off axis and labels
    plt.axis("off")
    plt.xticks([])
    plt.yticks([])

    # Plot the points as black dots
    plt.scatter(points[:, 0], points[:, 1], s=1, c="k")

    # Save the figure as an image
    plt.savefig(img, bbox_inches="tight", pad_inches=0, format="png")
    plt.show()